# Dashboard version 2

In [1]:
# Import required libraries
import copy
import pathlib
import dash
import math
import datetime as dt
import pandas as pd
import numpy as np
from dash.dependencies import Input, Output, State, ClientsideFunction
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from typing import Tuple
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from io import BytesIO
from wordcloud import WordCloud
import base64
import datetime
from scipy.stats import boxcox
from datetime import date
from scipy.special import inv_boxcox

In [10]:
JupyterDash.infer_jupyter_proxy_config()

In [3]:
# Load data
# get relative data folder
PATH = pathlib.Path('./data').parent
DATA_PATH = PATH.joinpath("data").resolve()
PIDAR = pd.read_csv(DATA_PATH.joinpath("Hisotrico de proyectos completo.csv"))
IPC = pd.read_csv(DATA_PATH.joinpath("data_ipc.csv"), sep = '|')
exportaciones = pd.read_csv(DATA_PATH.joinpath("exportaciones_cultivos.csv"), sep = '|')
SIPRA = pd.read_csv(DATA_PATH.joinpath("data_sipra_normalizado.csv"))
SIPRA['Indice']=SIPRA['Total aptitud [ha]']/SIPRA['Área departamento [ha]']
SIPRA_df = SIPRA.drop(SIPRA[SIPRA['Indice']>0.999].index)
SIP_df = SIPRA_df.pivot_table(index='Departamento', columns='Cultivo', values='Indice')

In [4]:
#Transformar data historica
def transform_hist(X1):
    
    scaler = StandardScaler()
    dff= X1.drop(columns = ['DEPARTAMENTO','MUNICIPIO', 'YEAR','CP','CICLO_CULTIVO']).copy()
    scaler.fit(dff)
    X2 = scaler.transform(dff)
    X3 =pd.DataFrame(X2, columns = dff.columns)
    X4 = pd.merge(X3,X1[['CP','CICLO_CULTIVO']],right_index=True,left_index=True)
    
    #Creating dummy variables
    cc_dummy = pd.get_dummies(X4['CICLO_CULTIVO'])
    cp_dummy = pd.get_dummies(X4['CP'])
    #Concatenating the dummy variables to the original dataset 
    cluster_dummy_set=pd.concat([X4,cc_dummy,cp_dummy],axis=1)
    #Deleting categorical variable from the dummy set
    del cluster_dummy_set['CICLO_CULTIVO']
    del cluster_dummy_set['CP']
    
    MCU = cluster_dummy_set.sample(frac = 1,random_state=42)
    MCL= cluster_dummy_set[~cluster_dummy_set.index.isin(MCU.index)]
    
    MCU=MCU.dropna()
    MCL=MCL.dropna()
    
    return MCU,MCL

#Obtener los cluster

def get_clusters(X_train: pd.DataFrame, X_test: pd.DataFrame, n_clusters: int) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    applies k-means clustering to training data to find clusters and predicts them for the test set
    """
    clustering = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward').fit(X_train)
    # apply the labels
    train_labels = clustering.labels_
    X_train_clstrs = X_train.copy()
    X_train_clstrs['clusters'] = train_labels
    
    # predict labels on the test set
    test_labels = clustering.fit_predict(X_train)
    X_test_clstrs = X_test.copy()
    X_test_clstrs['clusters'] = test_labels
    return X_train_clstrs, X_test_clstrs

# Crear la calificacion para cada cluster
def calificar(X_train_clstrs, X1):
    #Anexando el cluster a los valores en la escala real
    idx = X_train_clstrs.index
    X5 = X1.iloc[idx].copy()
    X5['key'] = idx
    X_train_clstrs['key']=  idx
    MCU_cl = pd.merge(X5, X_train_clstrs[['clusters','key']],on = 'key', how='left')
    
    #Claificacion de cada cluster
    cols = ['Total beneficiarios','Valor cofinanciación', 'Valor Contrapartida','RENDIMIENTO', 'PRODUCTIVIDAD']
    CALIF = pd.DataFrame()
    for i in cols:
        y = MCU_cl.groupby(['clusters'])[[i]].mean().sort_values(by = i, ascending = False).reset_index()
        y['cal'] = [4,3,2,1]
        y.columns = ['cluster',i,'cal_'+ str(i)]
        temp = y[['cluster','cal_'+ str(i)]]
        if len(CALIF) == 0:
            CALIF = temp
        else:
            CALIF = pd.merge(CALIF,temp,on='cluster',how ='inner')
            
    PROBABILIDADES = CALIF.iloc[:,[1,2,3,4]].dot(get_pesos())/4
    calificaciones = pd.merge(CALIF['cluster'],PROBABILIDADES,right_index=True,left_index=True)
    
    return calificaciones

def get_pesos():
    return [[0.3],[0.1],[0.2],[0.4]]

In [5]:
X_train, X_test = transform_hist(PIDAR)
X_train_clstrs, X_test_clstrs = get_clusters(X_train, X_test,4)
calif = calificar(X_train_clstrs,PIDAR)
PIDAR_df = pd.merge(PIDAR,X_train_clstrs[['clusters']],right_index=True,left_index=True)

In [6]:
#Cargar Geo Json
import json
with open('Colombia.geo.json') as f:
    counties = json.load(f)
#Cambiar los nombre de los proyectos del df acorde con el geojson
counties['features'][16]['properties']['NOMBRE_DPT'] = 'NARINO'
Departamentos = []
geocol = counties.copy()
for man in geocol['features']:
    Departamentos.append(man['properties']['NOMBRE_DPT'])
    man['id']=man['properties']['NOMBRE_DPT']

In [19]:
#Cargar la informacion e indicadores necesariso para el dash

##Cargar el logo del dashboard
test_png = './assets/my-image.png'
test_base64 = base64.b64encode(open(test_png, 'rb').read()).decode('ascii')

##Cargar la lista de años que tenemos disponible para el slider
YEARS = PIDAR['YEAR'].unique()
YEARS = np.sort(YEARS)

#Linea productiva
WELL_STATUSES = {'CAFE':'CAFE',
'PLATANO':'PLATANO',
'CAUCHO':'CAUCHO',
'TOMATE':'TOMATE',
'PAPAYA':'PAPAYA',
'YUCA':'YUCA',
'ARRACACHA':'ARRACACHA',
'PIÑA':'PIÑA',
'YACON':'YACON',
'CAÑA PANELERA':'CAÑA PANELERA',
'ARROZ':'ARROZ',
'LULO':'LULO',
'NARANJA':'NARANJA',
'PEPINO COHOMBRO':'PEPINO COHOMBRO',
'PIMENTON':'PIMENTON',
'TRIGO':'TRIGO',
'TOMATE DE ARBOL':'TOMATE DE ARBOL',
'CEBOLLA DE BULBO':'CEBOLLA DE BULBO',
'MAIZ FORRAJERO':'MAIZ FORRAJERO',
'MANGO':'MANGO',
'CIMARRON':'CIMARRON',
'CAÑA AZUCARERA':'CAÑA AZUCARERA',
'MORA':'MORA',
'PAPA':'PAPA',
'GRANADILLA':'GRANADILLA',
'HABA':'HABA',
'ALGODON':'ALGODON',
'CACAO':'CACAO',
'ÑAME':'ÑAME',
'MELON':'MELON',
'AJI':'AJI',
'MANDARINA':'MANDARINA',
'BANANO':'BANANO',
'PATILLA':'PATILLA',
'AGUACATE':'AGUACATE',
'CHONTADURO':'CHONTADURO',
'FRIJOL':'FRIJOL',
'REPOLLO':'REPOLLO',
'REMOLACHA':'REMOLACHA',
'CURUBA':'CURUBA',
'ULLUCO':'ULLUCO',
'CEBOLLA DE RAMA':'CEBOLLA DE RAMA',
'BROCOLI':'BROCOLI',
'CILANTRO':'CILANTRO',
'UVA':'UVA',
'LECHUGA':'LECHUGA',
'ARVEJA':'ARVEJA',
'LIMON':'LIMON',
'ARAZA':'ARAZA',
'COCO':'COCO',
'BOROJO':'BOROJO',
'MAIZ':'MAIZ',
'AHUYAMA':'AHUYAMA',
'GUANABANA':'GUANABANA',
'MARACUYA':'MARACUYA',
' ASAI':' ASAI',
'PITAHAYA':'PITAHAYA',
'FRESA':'FRESA',
'GUAYABA':'GUAYABA',
'MANGOSTINO':'MANGOSTINO',
'HORTALIZAS VARIAS':'HORTALIZAS VARIAS',
'ZANAHORIA':'ZANAHORIA',
'BADEA':'BADEA',
'MALANGA':'MALANGA',
'COLIFLOR':'COLIFLOR',
'PALMA DE ACEITE':'PALMA DE ACEITE',
'BREVO':'BREVO',
'FIQUE':'FIQUE',
'UCHUVA':'UCHUVA',
'TANGELO':'TANGELO',
'ASAI':'ASAI',
'FRESA':'FRESA',
}

#tipo de proyecto o cadena productiva
WELL_TYPES = {
'META':'META',
 'ANTIOQUIA': 'ANTIOQUIA',
 'CAUCA': 'CAUCA',
 'CALDAS': 'CALDAS',
 'HUILA': 'HUILA',
 'BOLIVAR': 'BOLIVAR',
 'CHOCO': 'CHOCO',
 'NORTE DE SANTANDER': 'NORTE DE SANTANDER',
 'VALLE DEL CAUCA': 'VALLE DEL CAUCA',
 'TOLIMA': 'TOLIMA',
 'RISARALDA': 'RISARALDA',
 'PUTUMAYO': 'PUTUMAYO',
'NARINO':'NARINO',
'CESAR':'CESAR',
'CAQUETA':'CAQUETA',
'BOYACA':'BOYACA',
'ATLANTICO':'ATLANTICO',
'ARAUCA':'ARAUCA',
'VICHADA':'VICHADA',
'VAUPES':'VAUPES',
'SUCRE':'SUCRE',
'SANTANDER':'SANTANDER',
'QUINDIO':'QUINDIO',
'MAGDALENA':'MAGDALENA',
'LA GUAJIRA':'LA GUAJIRA',
'GUAVIARE':'GUAVIARE',
'GUAINIA':'GUAINIA',
'CUNDINAMARCA':'CUNDINAMARCA',
'CORDOBA':'CORDOBA',
'CASANARE':'CASANARE',
 'AMAZONAS': 'AMAZONAS',
}

# Create controls
well_status_options = [
    {"label": str(WELL_STATUSES[well_status]), "value": str(well_status)}
    for well_status in WELL_STATUSES
]

well_type_options = [
    {"label": str(WELL_TYPES[well_type]), "value": str(well_type)}
    for well_type in WELL_TYPES
]

available_indicators = ["Productivity (%)","Beneficiaries (number of people)", "Performance (Tons/Ha)",
                        'Amount cofinanciated (Millions COP)','IPC (% Monthly Variability)', 'Exports (Thousands of tons)']
dic_ind = {"Productivity (%)":'PRODUCTIVIDAD',"Beneficiaries (number of people)":'Total beneficiarios', 
           "Performance (Tons/Ha)":"RENDIMIENTO",'Amount cofinanciated (Millions COP)':'Valor cofinanciación','IPC (% Monthly Variability)':'IPC',
          'Exports (Thousands of tons)':'Exports'}
clusters_indicators = ['Cluster High = Highest productivity and people beneficiated', 'Cluster Medium = High productivity but low people beneficiated',
                       'Cluster Low = Productivity and people beneficiated less than the mean' , 'Cluster No Productive= Low productivity and people beneficiated']

lab = {'PRODUCTIVIDAD': "Productivity (%)",
 'Total beneficiarios': "Beneficiaries (number of people)",
 "RENDIMIENTO": "Performance (Tons/Ha)",
 'Valor cofinanciación':'Amount cofinanciated (Millions COP)','IPC':'IPC (% Monthly Variability)', 'Exports':'Exports (Thousands of tons)'}

In [62]:
app = dash.Dash(__name__, meta_tags=[{"name": "viewport", "content": "width=device-width"}], suppress_callback_exceptions = True)
app.config['suppress_callback_exceptions'] = True
app.title = 'ADR' 
server = app.server

# Create global chart template
layout = dict(
    autosize=True,
    automargin=True,
    margin=dict(l=30, r=30, b=20, t=40),
    hovermode="closest",
    plot_bgcolor="#F9F9F9",
    paper_bgcolor="#F9F9F9",
    legend=dict(font=dict(size=10), orientation="h"),
    title="Satellite Overview",
    mapbox=dict(
        style="light",
        center=dict(lon=-78.05, lat=42.54),
        zoom=7,
    ),
)

# Create app layout
app.layout = html.Div(
    [
        dcc.Store(id="aggregate_data"),
        # empty Div to trigger javascript file for graph resizing
        html.Div(id="output-clientside"),
        html.Div(
            [
                html.Div(
                    [
                        html.Img(
                            src='data:image/png;base64,{}'.format(test_base64),
                            id="plotly-image",
                            style={
                                "height": "60px",
                                "width": "auto",
                                "margin-bottom": "25px",
                            },
                        )
                    ],
                    className="one-third column",
                ),
                html.Div(
                    [
                        html.Div(
                            [
                                html.H1(
                                    "PIDAR projects",
                                    style={"margin-bottom": "0px"},
                                ),
                                html.H5(
                                    "Comprehensive Agricultural Development Plans with a Territorial Approach \
                (PIDAR for its Spanish acronym)", style={"margin-top": "0px"}
                                ),
                            ]
                        )
                    ],
                    className="one-half column",
                    id="title",
                ),
                html.Div(
                    [
                        html.A(
                            html.Button("Learn More", id="learn-more-button"),
                            href="https://www.adr.gov.co/Paginas/Agencia-de-Desarrollo-Rural.aspx",target='_blank'
                        )
                    ],
                    className="one-third column",
                    id="button",
                ),
            ],
            id="header",
            className="row flex-display",
            style={"margin-bottom": "25px"},
        ),
        html.Div(
            [
                html.Div(
                    [
                        html.P(
                            "Filter by project year date:",
                            className="control_label",
                        ),
                        dcc.RangeSlider(
                            id="year_slider",
                            min=min(YEARS),
                            max=max(YEARS),
                            value=[min(YEARS), max(YEARS)],
                            className="dcc_control",
                            marks={
                                        str(year): {
                                            "label": str(year),
                                            "style": {"color": "#7fafdf"},
                                        }
                                        for year in YEARS
                                    }
                        ),
                        html.P("Filter by production chain line:", className="control_label"),
                        dcc.RadioItems(
                            id="well_status_selector",
                            options=[
                                {"label": "All ", "value": "all"},
                                {"label": "Customize ", "value": "custom"},
                            ],
                            value="all",
                            labelStyle={"display": "inline-block"},
                            className="dcc_control",
                        ),
                        dcc.Dropdown(
                            id="well_statuses",
                            options=well_status_options,
                            multi=True,
                            value=list(WELL_STATUSES.keys()),
                            className="dcc_control",
                        ),
                        html.P("Filter by department:", className="control_label"),
                        dcc.RadioItems(
                            id="well_type_selector",
                            options=[
                                {"label": "All ", "value": "all"},
                                {"label": "Customize ", "value": "custom"},
                            ],
                            value="all",
                            labelStyle={"display": "inline-block"},
                            className="dcc_control",
                        ),
                        dcc.Dropdown(
                            id="well_types",
                            options=well_type_options,
                            multi=True,
                            value=list(WELL_TYPES.keys()),
                            className="dcc_control",
                        ),
                    ],
                    className="pretty_container four columns",
                    id="cross-filter-options",
                ),
                html.Div(
                    [
                        html.Div(
                            [
                                html.Div(
                                    [html.H6(id="well_text"), html.P("No. of projects")],
                                    id="wells",
                                    className="mini_container",
                                ),
                                html.Div(
                                    [html.H6(id="gasText"), html.P("Beneficiaries (number of people)")],
                                    id="gas",
                                    className="mini_container",
                                ),
                                html.Div(
                                    [html.H6(id="oilText"), html.P('Performance (tons/ha)')],
                                    id="oil",
                                    className="mini_container",
                                ),
                                html.Div(
                                    [html.H6(id="waterText"), html.P("Amount co-financed (Millions COP)")],
                                    id="water",
                                    className="mini_container",
                                ),
                            ],
                            id="info-container",
                            className="row container-display",
                        ),
                        html.Div(
                            [html.H5("Distribution of the financial resources per department in the time", 
                                     style={'font-weight': 'bold','textAlign': 'center'}),
                             html.P("Here you can see the distribution of the money for the projects among departments",
                             style={'textAlign': 'center'}),
                             dcc.Graph(id="count_graph")],
                            id="countGraphContainer",
                            className="pretty_container",
                        ),
                    ],
                    id="right-column",
                    className="eight columns",
                ),
            ],
            className="row flex-display",
        ),
        html.Div([
                        html.H6("""Our project aims to determine which productive chains should be financed 
                        in the 2021, and for this purpose we classificate the project history in clusters as 
                        it can be see bellow""",style={'textAlign': 'center'}),
                    ],
                    style={"margin-bottom": "25px",'textAlign': 'center','font-weight': 'italic'},
                    id='middle'
                ),
        html.Div(
            [
                html.Div(
                    [html.H5("Classification of PIDAR projects",style={'font-weight': 'bold','textAlign': 'center'}),
                     html.P("""According to the conditions of the passed projects this is 
                     the classification that we performed""",style={'textAlign': 'center'},
                            className="control_label",
                        ),
                     dcc.Dropdown(id='cluster_slider',
                        options=[{'label': j, 'value': i} for j, i in zip(clusters_indicators, range(4))],
                        value=0),
                     html.Br(),
                    dcc.Graph(id="main_graph", hoverData={'points': [{'customdata': 'PLATANO'}]}), 
                    ],
                    className="pretty_container seven columns",
                ),
                html.Div(
                    [html.H5("Time series per variable",style={'font-weight': 'bold','textAlign': 'center'}),
                     html.P("""Select the variable to analyse by productive chain: """,style={'textAlign': 'center'},
                            className="control_label",
                        ),
                     dcc.Dropdown(id='crossfilter-xaxis-column',
                        options=[{'label': i, 'value': i} for i in available_indicators],
                        value='IPC (% Monthly Variability)'),
                     dcc.Graph(id="individual_graph")],
                    className="pretty_container five columns",
                ),
            ],
            className="row flex-display",
        ),
        html.Div(
            [
                html.Div(
                    [html.H5("Aptitude for each productive chain per department",style={'font-weight': 'bold','textAlign': 'center'}),
                     html.P("""Here you can see index aptitude for productive chain in each departments
                     Note: the higher the area in each column, the higher the aptitude for department""",style={'textAlign': 'center'},
                            className="control_label",
                        ),
                        dcc.Graph(id="pie_graph")],
                    
                    className="pretty_container seven columns",
                ),
                html.Div(
                    [html.H5("Distribution of the productive chain per Department",style={'font-weight': 'bold','textAlign': 'center'}),
                     html.P("""Here you can see the number of projects for each department
                     according to the productive chain selected in the cluster""",style={'textAlign': 'center'}),
                        dcc.Graph(id="aggregate_graph")],
                    className="pretty_container five columns",
                ),
            ],
            className="row flex-display",
        ),
    ],
    id="mainContainer",
    style={"display": "flex", "flex-direction": "column"},
)


# Helper functions
def human_format(num):
    if num == 0:
        return "0"

    magnitude = int(math.log(num, 1000))
    mantissa = str(int(num / (1000 ** magnitude)))
    return mantissa + ["", "M", "G", "T", "P"][magnitude]


def filter_dataframe(df, well_statuses, well_types, year_slider):
    df = df.replace(to_replace='NARIÃ‘O', value='NARINO', regex=True)
    dff = df[
        (df['CP'].isin(well_statuses))
        & (df['DEPARTAMENTO'].isin(well_types))
        & (df['YEAR'] >= year_slider[0])
        & (df['YEAR'] <= year_slider[1])]
    return dff

def filter_dataframe2(df, well_types):
    df = df.replace(to_replace='NARIÃ‘O', value='NARINO', regex=True)
    dff = df[
        (df['Departamento'].isin(well_types))]
    return dff

def produce_individual(api_well_num):
    try:
        points[api_well_num]
    except:
        return None, None, None, None

    index = list(
        range(min(points[api_well_num].keys()), max(points[api_well_num].keys()) + 1))
    gas = []
    oil = []
    water = []

    for year in index:
        try:
            gas.append(points[api_well_num][year]["Gas Produced, MCF"])
        except:
            gas.append(0)
        try:
            oil.append(points[api_well_num][year]["Oil Produced, bbl"])
        except:
            oil.append(0)
        try:
            water.append(points[api_well_num][year]["Water Produced, bbl"])
        except:
            water.append(0)

    return index, gas, oil, water


def produce_aggregate(dff):
    
    index = len(dff)
    gas = sum(dff['Total beneficiarios'])
    oil = round(sum(dff['RENDIMIENTO'].fillna(0)),0)
    water = round(sum(dff['Valor cofinanciación'])/1e6,0)

    return index, gas, oil, water

def time_series(selected, year_slider):
    
    index = list(range(max(year_slider[0], 2012), 2019))
    gas = []
    oil = []
    water = []

    for year in index:
        count_gas = 0
        count_oil = 0
        count_water = 0
        for api_well_num in selected:
            try:
                count_gas += PIDAR_df[api_well_num][year]['Total beneficiarios']
            except:
                pass
            try:
                count_oil += PIDAR_df[api_well_num][year]['Valor cofinanciación']
            except:
                pass
            try:
                count_water += PIDAR_df[api_well_num][year]['RENDIMIENTO']
            except:
                pass
        gas.append(count_gas)
        oil.append(count_oil)
        water.append(count_water)

    return index, gas, oil, water

def plot_wordcloud(data):
    d = {a: x for a, x in data.values}
    wc = WordCloud(background_color='white', width=540, height=300)
    wc.fit_words(d)
    return wc.to_image()

#Crear callbacks

# Boton de todas las cadenas productivas
@app.callback(Output("well_statuses", "value"), [Input("well_status_selector", "value")])

def display_status(selector):
    if selector == "all":
        return list(WELL_STATUSES.keys())
    return []

# Boton de todos los departamentos
@app.callback(Output("well_types", "value"), [Input("well_type_selector", "value")])
def display_type(selector):
    if selector == "all":
        return list(WELL_TYPES.keys())
    return []

#Listas de filtros
@app.callback(Output("aggregate_data", "data"),[
        Input("well_statuses", "value"),
        Input("well_types", "value"),
        Input("year_slider", "value"),
    ],
)
def update_production_text(well_statuses, well_types, year_slider):

    dff = filter_dataframe(PIDAR_df, well_statuses, well_types, year_slider)
    index, gas, oil, water = produce_aggregate(dff)
    return [gas, oil, water]


# Selectors -> well text
@app.callback(
    Output("well_text", "children"),
    [
        Input("well_statuses", "value"),
        Input("well_types", "value"),
        Input("year_slider", "value"),
    ],
)
def update_well_text(well_statuses, well_types, year_slider):
    df3 = filter_dataframe(PIDAR_df, well_statuses, well_types, year_slider)
    return df3.shape[0]


#Minicontainers
@app.callback(
    [
        Output("gasText", "children"),
        Output("oilText", "children"),
        Output("waterText", "children"),
    ],
    [Input("aggregate_data", "data")],
)
def update_text(data):
    return data[0], data[1], data[2]


#Map graph

@app.callback(
    Output("count_graph", "figure"),
    [Input("well_statuses", "value"),
     Input("well_types", "value"),
     Input("year_slider", "value")
    ]
)
        
def update_graph(well_statuses, well_types, year_slider):
        dff = filter_dataframe(PIDAR_df, well_statuses, well_types, year_slider)
        dff = dff.groupby('DEPARTAMENTO').sum().reset_index()
        fig = px.choropleth_mapbox(dff,                                      #Data
                                   locations='DEPARTAMENTO',                              #Column containing the identifiers used in the GeoJSON file 
                                   color='Valor cofinanciación',                        #Column giving the color intensity of the region
                                   geojson=geocol,                                        #The GeoJSON file
                                   zoom=4,                                                #Zoom
                                   mapbox_style="carto-positron",                         #Mapbox style, for different maps you need a Mapbox account and a token
                                   range_color=(0, dff['Valor cofinanciación'].max()), 
                                   center={"lat":4.7110, "lon": -74.0721},                #Center
                                   color_continuous_scale="Viridis",                      #Color Scheme
                                   opacity=0.5, #Opacity of the map
                                   labels={'Valor cofinanciación':'Amount cofinanciated (Millions COP)',
                                          'DEPARTAMENTO':'Department'},)
        
        fig.update_layout(autosize = True, transition_duration=500, margin=dict(l=20, r=20, t=2, b=65),
                          paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)')
        
        return fig
    
#Grafico Scatter
@app.callback(Output("main_graph", "figure"),[
        Input("well_statuses", "value"),
        Input("well_types", "value"),
        Input("year_slider", "value"),
        Input("cluster_slider", "value"),
    ]
)
def make_main_figure(well_statuses, well_types, year_slider, cluster_slider):
    dff = filter_dataframe(PIDAR_df, well_statuses, well_types, year_slider)
    dff = dff[dff['clusters'] == cluster_slider]
    #fig2 = px.scatter_3d(dff, x='PRODUCTIVIDAD', y='Total beneficiarios', z='RENDIMIENTO', color='clusters',hover_data=['DEPARTAMENTO','CP','YEAR'])
    fig2 = px.scatter_3d(dff, x=dff['RENDIMIENTO'], y=dff['Total beneficiarios'], z=dff['PRODUCTIVIDAD'], 
                         hover_name=dff['CP'], labels={
                     'PRODUCTIVIDAD': "Productivity (%)",
                     'Total beneficiarios': "Beneficiaries (number of people)",
                     "RENDIMIENTO": "Performance (Tons/Ha)"
                 })
    fig2.update_traces(customdata=dff['CP'])
    fig2.update_layout(legend=dict(orientation="h", 
                                    yanchor="bottom",
                                    y=1.02,
                                    xanchor="right",
                                    x=1),
                        paper_bgcolor='rgba(0,0,0,0)',
                        plot_bgcolor='rgba(0,0,0,0)',
                       margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

    return fig2

#Barras de departamento por cultivo
@app.callback(Output("aggregate_graph", "figure"), [Input("crossfilter-xaxis-column", "value"),
                                                    Input("well_statuses", "value"),
                                                    Input("well_types", "value"),
                                                    Input("year_slider", "value"),
                                                    Input('main_graph', 'hoverData'),
                                                    Input("cluster_slider", "value")])

def make_aggregate_figure(crossfilter,well_statuses, well_types, year_slider,hoverData,cluster_slider):
    if (len(well_statuses) > 0) and (len(well_types) > 0):
        dff = filter_dataframe(PIDAR_df, well_statuses, well_types, year_slider)
        result =  any(elem in dff['CP'].unique()  for elem in well_statuses)
        
        if result:
            temp = dic_ind["Productivity (%)"]
            country_name = hoverData['points'][0]['customdata']
            dff = dff[dff['clusters'] == cluster_slider].copy()
            dff4 = dff[dff['CP']==country_name].groupby(['DEPARTAMENTO'])[[temp]].count().sort_values(by=temp,ascending=False).head(5).reset_index()
            fig3 = px.bar(dff4.sort_values(by=temp,ascending=True), y=temp, x='DEPARTAMENTO', orientation='v', height=400,labels={'PRODUCTIVIDAD': 'Number of projects',
                                                                                                 'Total beneficiarios': 'Number of projects',
                                                                                                 "RENDIMIENTO": 'Number of projects',
                                                                                                'DEPARTAMENTO':'Department'})
            
            fig3.update_layout(showlegend=False,paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
            title = '<b>{}</b>'.format(country_name)
            fig3.add_annotation(x=0, y=0.85, xanchor='left', yanchor='bottom',
                                xref='paper', yref='paper', showarrow=False, align='right'
                                , text=title)
            
            return fig3
        
        else:
            return go.Figure()
            
    else:
        return go.Figure()
    
#Grafico time series
@app.callback(Output('individual_graph', "figure"),[Input("crossfilter-xaxis-column", "value"),
                                                    Input('main_graph', 'hoverData'),
                                                    Input("well_statuses", "value"),
                                                    Input("well_types", "value"),
                                                    Input("year_slider", "value"),
                                                    Input("cluster_slider", "value"),],)

def make_individual_figure(crossfilter, hoverData, well_statuses, well_types, year_slider, cluster_slider):
    if (len(well_statuses) > 0) and (len(well_types) > 0):
        valor = dic_ind[crossfilter]
        dff = filter_dataframe(PIDAR_df, well_statuses, well_types, year_slider)
        result =  any(elem in dff['CP'].unique()  for elem in well_statuses)
        
        if result:
            country_name = hoverData['points'][0]['customdata']
            dff2 = dff[dff['clusters'] == cluster_slider]

            if country_name in dff2['CP'].unique():
                
                if (crossfilter != 'IPC (% Monthly Variability)') and (crossfilter != 'Exports (Thousands of tons)'):
                    dff5 = dff2.groupby(['CP','YEAR'])[valor].mean()[country_name].reset_index()
                    fig4 = go.Figure()
                    fig4.add_trace(go.Scatter(x=dff5['YEAR'], y=dff5[valor], mode='lines+markers', name=country_name))
                    fig4.update_layout(xaxis_title='Year', yaxis_title=lab[valor],
                                      paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
                        
                    title = '<b>{}</b><br>{}'.format(country_name, crossfilter)
                    fig4.add_annotation(x=0, y=0.85, xanchor='left', yanchor='bottom',
                               xref='paper', yref='paper', showarrow=False, align='left',
                               bgcolor='rgba(255, 255, 255, 0.5)', text=title)

                    return fig4
                
                elif crossfilter == 'IPC (% Monthly Variability)':
                    df = IPC.copy()
                    df = df[df['cultivo']==country_name].copy()
                    df['mes_anio_1'] = pd.to_datetime(df['mes_anio_1'])

                    trace = go.Scatter(
                        name='Real value IPC',
                        x=df['mes_anio_1'],
                        y=df['valor_real'],
                        mode='lines+markers',line_color='#6666FF')

                    trace2 = go.Scatter(
                        name='Without Covid',
                        x=df[df['mes_anio_1']>"2020-04-01"]['mes_anio_1'],
                        y=df[df['mes_anio_1']>"2020-04-01"]['promedio_ipc_sin_cv'],
                        mode='lines+markers',line_color='#195d19')

                    trace3 = go.Scatter(
                        name='With Covid',
                        x=df[df['mes_anio_1']>"2020-04-01"]['mes_anio_1'],
                        y=df[df['mes_anio_1']>"2020-04-01"]['promedio_ipc_con_cov'],
                        mode='lines+markers',line_color='#ad3232')

                   # Trace order can be important
                    # with continuous error bars
                    data = [trace, trace2,trace3]

                    layout = go.Layout(
                        yaxis=dict(title='Monthly Variability (%)'),
                        xaxis=dict(title='Month per Year'),
                        legend=dict(
                        orientation="h",
                        yanchor="bottom",
                        y = -0.6,
                        xanchor="right",
                        x=1),
                        paper_bgcolor='rgba(0,0,0,0)',
                        plot_bgcolor='rgba(0,0,0,0)')

                    fig6 = go.Figure(data=data, layout=layout)
                    title = '<b>{}</b><br>{}'.format(country_name, crossfilter)
                    fig6.add_annotation(x=0, y=0.85, xanchor='left', yanchor='bottom',
                                        xref='paper', yref='paper', showarrow=False, align='left',
                                        bgcolor='rgba(255, 255, 255, 0.5)', text=title)
                    
                    return fig6
                
                else:
                    df_exp = exportaciones.copy()
                    df_exp = df_exp[df_exp['cadena_cultivo']==country_name].copy()
                    df_exp['mes_anio_1'] = pd.to_datetime(df_exp['mes_anio1'])

                    trace = go.Scatter(
                        name='Real Export Value',
                        x=df_exp['mes_anio_1'],
                        y=df_exp['valor_real'],
                        mode='lines+markers',line_color='#6666FF')

                    trace2 = go.Scatter(
                        name='Normal Scenario forecast',
                        x=df_exp[df_exp['mes_anio_1']>"2019-12-01"]['mes_anio_1'],
                        y=df_exp[df_exp['mes_anio_1']>"2019-12-01"]['promedio_exportaciones'],
                        mode='lines+markers',line_color='#195d19')

                    upper_bound = go.Scatter(
                        name='Upper Stage',
                        x=df_exp[df_exp['mes_anio_1']>"2019-12-01"]['mes_anio_1'],
                        y=df_exp[df_exp['mes_anio_1']>"2019-12-01"]['promedio_exportaciones_superior'],
                        mode='lines',
                        marker=dict(color="#444"),
                        line=dict(width=0),
                        fillcolor='rgba(68, 68, 68, 0.3)',
                        fill='tonexty')

                    lower_bound = go.Scatter(
                        name='Lower Stage',
                        x=df_exp[df_exp['mes_anio_1']>"2019-12-01"]['mes_anio_1'],
                        y=df_exp[df_exp['mes_anio_1']>"2019-12-01"]['promedio_exportaciones_inferior'],
                        marker=dict(color="#444"),
                        line=dict(width=0),
                        mode='lines',
                        fillcolor='rgba(68, 68, 68, 0.3)',
                        fill='tonexty')

                    # Trace order can be important
                    # with continuous error bars
                    data = [trace, trace2,lower_bound,upper_bound]

                    layout = go.Layout(
                        yaxis=dict(title='Number of quantity of exports(Thousands)'),
                        xaxis=dict(title='Month per Year'),
                        legend=dict(
                        orientation="h",
                        yanchor="bottom",
                        y = -0.6,
                        xanchor="right",
                        x=1),
                        paper_bgcolor='rgba(0,0,0,0)',
                        plot_bgcolor='rgba(0,0,0,0)')

                    fig7 = go.Figure(data=data, layout=layout)
                    title = '<b>{}</b><br>{}'.format(country_name, crossfilter)
                    fig7.add_annotation(x=0, y=0.85, xanchor='left', yanchor='bottom',
                                        xref='paper', yref='paper', showarrow=False, align='left',
                                        bgcolor='rgba(255, 255, 255, 0.5)', text=title)
                    
                    return fig7
            
            
            else:
                return go.Figure()
        
        else:
            return go.Figure()
        
    else:
        return go.Figure()


@app.callback(Output("pie_graph", "figure"),[Input("well_statuses", "value"),
                                             Input("well_types", "value"),
                                             Input("year_slider", "value"),
                                             Input('main_graph', 'hoverData'),
                                             Input("cluster_slider", "value"),])

def make_pie_figure(well_statuses, well_types, year_slider,hoverData,cluster_slider):
    if (len(well_statuses) > 0) and (len(well_types) > 0):
        dff = filter_dataframe(PIDAR_df, well_statuses, well_types, year_slider)
        result =  any(elem in dff['CP'].unique()  for elem in well_statuses)
        if result:
            country_name = hoverData['points'][0]['customdata']
            dff = dff[dff['clusters'] == cluster_slider].copy()
            dff2 = dff[dff['CP']==country_name].groupby(['DEPARTAMENTO'])[["PRODUCTIVIDAD"]].count().sort_values(by="PRODUCTIVIDAD",ascending=False).head(5).reset_index()
            list_dp = dff2['DEPARTAMENTO'].unique()
            dfff = filter_dataframe2(SIPRA, well_types)
            result2 =  any(elem in dfff['Departamento'].unique()  for elem in list_dp)
            if result2:
                dfff4 = dfff[dfff['Departamento'].isin(list_dp)].copy()
                fig5 = px.bar(dfff4, x= 'Cultivo',y='Indice',color='Departamento',
                              labels={'Cultivo':'Productive chain',
                                     'Indice':'Index aptitude (area available/total area)'})
                fig5.update_layout(legend=dict(
                    orientation="h",
                    yanchor="bottom",
                    y = -0.6,
                    xanchor="right",
                    x=1), paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
        
                return fig5
            
            else:
                return go.Figure()
        
        else:
            return go.Figure()
    
    else:
        return go.Figure()


In [ ]:
app.run_server(debug=True, use_reloader=False)

Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Debugger PIN: 445-626-426
Debugger PIN: 445-626-426
Debugger PIN: 445-626-426
Debugger PIN: 445-626-426
Debugger PIN